In [54]:
%pip install pytesseract
%pip install python-dateutil
# Install tesseract exe also from link -> https://github.com/tesseract-ocr/tesseract?tab=readme-ov-file#installing-tesseract


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: python-dateutil in d:\development\python-mini-projects\date_extractor\venv\lib\site-packages (2.9.0.post0)




[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pytesseract
from PIL import Image
import re
from dateutil import parser
import os

In [2]:
def get_date(img):
    ocr_text = pytesseract.image_to_string(img)
    img = Image.open(img)
    date_regexes = [
        r'\d{1,2}[./-]\d{1,2}[./-]\d{2,4}',  # e.g., 01/31/2024
        r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{2,4}',
    ]

    hour_regex = r'\d{1,2}'
    minute_regex = r':\d{2}'
    ampm_regex = r'AM|PM'

    extracted_dates = []
    for regex in date_regexes:
        matches = re.findall(regex, ocr_text)
        for match in matches:
            extracted_dates.append(match)

    parsed_dates = []
    for date_str in extracted_dates:
        try:
            parsed_date = parser.parse(date_str, fuzzy=True)
            parsed_dates.append(parsed_date)
        except ValueError:
            pass

    hour_match = re.search(hour_regex, ocr_text)
    minute_match = re.search(minute_regex, ocr_text)
    ampm_match = re.search(ampm_regex, ocr_text)

    hour = hour_match.group() if hour_match else None
    minute = minute_match.group()[1:] if minute_match else None
    ampm = ampm_match.group() if ampm_match else None

    formatted_dates = [date.strftime("%d %b %Y") for date in parsed_dates]

    combined_datetime = []
    for date in formatted_dates:
        combined_datetime.append(f"{date} {hour}:{minute} {ampm}")

    final_date = ""
    for dt in combined_datetime:
        # print(dt)
        final_date = dt
    
    return final_date

In [3]:
# Path to the folder containing images
folder_path = "./sample"

# Create or open a text file to write the results
output_file_path = "./output/extracted_dates.txt"
with open(output_file_path, "w") as output_file:
    files = os.listdir(folder_path)

    for file in files:
        if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
            image_path = os.path.join(folder_path, file)
            date = get_date(image_path)
            if date is None or date == "":
                output_file.write(f"Image: {file}, Date: Image quality low\n")
            else:
                output_file.write(f"Image: {file}, Date: {date}\n")

print("Extraction completed. Results written to:", output_file_path)

Extraction completed. Results written to: ./output/extracted_dates.txt


## Shaja code 1

In [2]:
import pytesseract
from PIL import Image
import os
import re

# Specify the path to the Tesseract executable
# For Windows, it's something like 'C:/Program Files/Tesseract-OCR/tesseract.exe'
# For Linux, it might be just 'tesseract' if it's already in the PATH
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

# Function to extract text from an image
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='eng')
    return text

# Function to find dates in text
def find_dates_in_text(text):
    # Regular expression for matching various date formats (e.g., DD/MM/YYYY, MM/DD/YYYY, etc.)
    # This is a basic pattern and might need to be adjusted or expanded depending on the date formats encountered
    date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{2,4}[-/]\d{1,2}[-/]\d{1,2})\b'
    dates = re.findall(date_pattern, text)
    return dates

# Specify your folder path containing the image files
folder_path = './sample'

# Process each file in the folder
output_file_path = "./output/extracted_dates_code1.txt"
with open(output_file_path, "w") as output_file:
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Check for common image file extensions
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)
            dates = find_dates_in_text(extracted_text)
            # print(f"Dates found in {filename}: {dates}")
            output_file.write(f"Image: {filename}, Date: {dates}\n")

print("Extraction completed. Results written to:", output_file_path)

Extraction completed. Results written to: ./output/extracted_dates_code1.txt


# Shaja code 2

In [3]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import os
import re

# Specify the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def preprocess_image_for_ocr(image_path):
    # Open the image
    image = Image.open(image_path)
    # Apply image pre-processing to enhance OCR recognition
    image = image.convert('L')  # Convert to grayscale
    image = image.filter(ImageFilter.MedianFilter())  # Apply a median filter for noise reduction
    enhancer = ImageEnhance.Contrast(image)  # Enhance image contrast
    image = enhancer.enhance(2)
    image = image.filter(ImageFilter.SHARPEN)  # Sharpen the image
    return image

def extract_text_from_image(image):
    text = pytesseract.image_to_string(image, lang='eng')
    return text

def find_dates_in_text(text):
    date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{2,4}[-/]\d{1,2}[-/]\d{1,2})\b'
    dates = re.findall(date_pattern, text)
    return dates

folder_path = './sample'

output_file_path = "./output/extracted_dates_code2.txt"
with open(output_file_path, "w") as output_file:
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image_for_ocr(file_path)
            extracted_text = extract_text_from_image(preprocessed_image)
            dates = find_dates_in_text(extracted_text)
            # print(f"Dates found in {filename}: {dates}")
            output_file.write(f"Image: {filename}, Date: {dates}\n")

print("Extraction completed. Results written to:", output_file_path)

Extraction completed. Results written to: ./output/extracted_dates_code2.txt


# Shaja code 3

In [5]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import os
import re

# Load the pre-trained model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

def extract_text_from_image(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    output = model.generate(pixel_values)
    text = processor.batch_decode(output, skip_special_tokens=True)[0]
    return text

def find_dates_in_text(text):
    date_pattern = r'\b(?:\d{1,2}[-/]\d{1,2}[-/]\d{2,4}|\d{2,4}[-/]\d{1,2}[-/]\d{1,2})\b'
    dates = re.findall(date_pattern, text)
    return dates

folder_path = './sample'

output_file_path = "./output/extracted_dates_code3.txt"
with open(output_file_path, "w") as output_file:
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            extracted_text = extract_text_from_image(file_path)
            dates = find_dates_in_text(extracted_text)
            # print(f"Dates found in {filename}: {dates}")
            output_file.write(f"Image: {filename}, Date: {dates}\n")

print("Extraction completed. Results written to:", output_file_path)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


KeyboardInterrupt: 